实现一下多输入通道互相关运算（互相关运算和卷积很相似）

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K)) # 在0维度遍历，把0维度的每个元素取出来（是个矩阵），进行互相关运算

验证互相关运算的输出

In [11]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]]) # [2, 3, 3]
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]]) # [2, 2, 2]

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多个通道的输出的互相关函数

In [12]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [14]:
corr2d_multi_in_out(X, K) # 多输出通道，实际上就是进行了多次的单输出通道罢了

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1x1卷积（实际上就是全连接层罢了，作用是：融合不同通道的特征）

In [19]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape # 输入的图片一般是：通道数、高、宽
    c_o = K.shape[0] # 输出通道数、输入通道数、核高、核宽
    X = X.reshape((c_i, h * w)) # 将高和宽拉成向量
    K = K.reshape((c_o, c_i)) # 因为1*1的卷积 核高、核宽都为1，因此直接reshape0维度和1维度罢了
    Y = torch.matmul(K, X) # 进行矩阵乘法
    return Y.reshape((c_o, h, w)) # 恢复高和宽

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6 # 不满足条件，则抛出异常
Y1==Y2

tensor([[[True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True]]])